In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard
%matplotlib inline

In [2]:
from utils import *
from scale_layer import *
from datetime import date

In [3]:
trainloader, testloader, classes = load_CIFAR()

Files already downloaded and verified
Files already downloaded and verified


### Batchnorm Training
- `model_bn` is the model with batchnorm layers
- train this model on the CIFAR 10 dataset

In [4]:
loss_history_bn = []
writer_bn = SummaryWriter(log_dir='runs/bn_stats-relu-test-%s' % date.today(), flush_secs=10)

In [5]:
model_bn = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    ScaleLayer(writer=writer_bn),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    ScaleLayer(writer=writer_bn),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    ScaleLayer(writer=writer_bn),
    nn.Linear(1152, 512),
    ScaleLayer(writer=writer_bn),
    nn.Linear(512, 128),
    ScaleLayer(writer=writer_bn),
    nn.Linear(128, 10)
)

In [6]:
ce_loss = nn.CrossEntropyLoss(reduction='sum')
def criterion(op, y, model_p):
    params = list(model_p.parameters())
    return ce_loss(op, y) #+ (1/len(params))*(torch.norm(torch.cat([param.view(-1) for param in params]))) #+ torch.sum((1/len(params))*torch.exp(torch.abs(torch.cat([param.view(-1) for param in params]))))


In [7]:
loss_tanh = train_model(model_bn, trainloader, criterion, optimizer = optim.Adam(list(model_bn.parameters()), lr=3e-4), epochs=20, writer=writer_bn)

[1,   300] loss: 112.427
[1,   600] loss: 103.488
[2,   300] loss: 114.100
[2,   600] loss: 98.359
[3,   300] loss: 84.469
[3,   600] loss: 89.483
[4,   300] loss: 93.660
[4,   600] loss: 82.782
[5,   300] loss: 73.620
[5,   600] loss: 60.956
[6,   300] loss: 93.441
[6,   600] loss: 60.695
[7,   300] loss: 79.152
[7,   600] loss: 73.166
[8,   300] loss: 55.102
[8,   600] loss: 75.242
[9,   300] loss: 73.722
[9,   600] loss: 56.814
[10,   300] loss: 62.010
[10,   600] loss: 62.190
[11,   300] loss: 58.661
[11,   600] loss: 62.976
[12,   300] loss: 60.812
[12,   600] loss: 50.407
[13,   300] loss: 73.072
[13,   600] loss: 47.746
[14,   300] loss: 53.078
[14,   600] loss: 63.372
[15,   300] loss: 54.213
[15,   600] loss: 43.194
[16,   300] loss: 53.638
[16,   600] loss: 52.315
[17,   300] loss: 46.938
[17,   600] loss: 45.979
[18,   300] loss: 44.667
[18,   600] loss: 43.189
[19,   300] loss: 30.940
[19,   600] loss: 39.713
[20,   300] loss: 45.075
[20,   600] loss: 44.602


In [8]:
stats_bn = print_stats(model_bn)

Mean:  0.020245323
Standard Deviation:  0.027986802
Sparsity:  0.00492764904473761


In [9]:
bn_model_err = get_model_error(model_bn, testloader)

test error: 0.3124
